# Waves Approaching a Shoreline

In [ ]:
import fastfd as ffd
ffd.sparse_lib('scipy')

import holoviews as hv
hv.extension('bokeh')

import numpy as np
import time

In [ ]:
length = 50 # wave length
amplitude = 1.25 # wave amplitude
spatial_acc = 6 # spacial derivative accuracy
time_acc = 2 # time derivative accuracy
timestep = 0.1 # timestep

g = 9.81 # gravitational constant

# correlations for wave velocity vs bottom depth
transition_speed = lambda l, d: np.sqrt(g * l / (2 * np.pi) * np.tanh(2 * np.pi * d / l))
shallow_speed = lambda d: np.sqrt(g/d)
wave_speed = lambda l, d: np.maximum(transition_speed(l, d), shallow_speed(d))

In [ ]:
# model definition
x = ffd.LinearAxis('x', start = 0, stop = 500, num = 201)
y = ffd.LinearAxis('y', start = 0, stop = 500, num = 201)

u = ffd.Scalar('u', [x, y], accuracy = 4)

model = ffd.FDModel([u], timestep = timestep)

In [ ]:
# define the depth of the bottom
def depth_model(x_, y_):
    x_bottom = -(x_ * x.delta - 250) / 100
    bottom = (x_bottom / np.sqrt(1 + x_bottom**2) + 1) * 5
    
    bump_1_x = (x_ * x.delta - 150) / 150
    bump_1_y = (y_ * y.delta - 150) / 150
    bump_1_r = np.sqrt(bump_1_x**2 + bump_1_y**2)
    bump_1 = -4 * (bump_1_r**2 + 1)**(-3/2)
    
    bump_2_x = (x_ * x.delta - 250) / 100
    bump_2_y = (y_ * y.delta - 325) / 100
    bump_2_r = np.sqrt(bump_2_x**2 + bump_2_y**2)
    bump_2 = -3.3 * (bump_2_r**2 + 1)**(-3/2)
    
    hole_x = (x_ * x.delta - 350) / 100
    hole_y = (y_ * y.delta - 225) / 100
    hole_r = np.sqrt(hole_x**2 + hole_y**2 - hole_x *hole_y )
    hole = 2 * (hole_r**2 + 1)**(-3/2)
    
    return bottom + bump_1 + bump_2 + hole

In [ ]:
# plot shoreline depth and wave velocity
depth = np.fromfunction(depth_model, (x.num_points, y.num_points))
velocity = wave_speed(length, depth)

depth_plot = hv.Image(
    depth[:, ::-1].T,
    bounds = (0, 0, 500 ,500)
).opts(
    width = 450, height = 425,
    tools = ['hover'], colorbar = True,
    title = ('Depth (m)')
)

depth_contours = hv.operation.contours(depth_plot).opts(cmap = 'greys_r', alpha = 0.5)

velocity_plot = hv.Image(
    velocity[:, ::-1].T,
    bounds = (0, 0, 500 ,500)
).opts(
    cmap = 'hot',
    width = 450, height = 425,
    tools = ['hover'], colorbar = True,
    title = ('Wave Velocity (m/s)')
)

velocity_contours = hv.operation.contours(velocity_plot).opts(cmap = 'greys', alpha = 0.5)

hv.Layout([
    hv.Overlay([depth_plot, depth_contours]).opts(show_legend = False),
    hv.Overlay([velocity_plot, velocity_contours]).opts(show_legend = False)
]).opts(shared_axes = False)

In [ ]:
# calculate average velocity at x=0 and wave period 
vel_avg = velocity[0].mean()
period = length / vel_avg
period

In [ ]:
# define model governing equations (wave equation in 2D)
model.update_equations({
    'u': (velocity.ravel()**2 * (u.d('x', derivative = 2) + u.d('y', derivative = 2)) \
          - u.dt('lhs', derivative = 2, accuracy = time_acc),
          None),
})

# apply some boundary condiditons
model.update_bocos({
    'ocean': (u.i[0, :], u.i[0, :], None),
    'shore': (u.i[-1, :], u.d('x')[-1, :], 0),
    'edges': (u.i[:, [0,-1]], u.d('y')[:, [0,-1]], 0), # Not ideal - should be absorptive
})

# precalculate transient rhs terms to minimize loop overhead
dt_const = u.dt('rhs', derivative = 2, accuracy = time_acc)

# model iteration updates
def update_timestep(u0, t):
    model.update_equations({
        'u': (None, dt_const * u0.ravel()),
    })

    model.update_bocos({
        'ocean': (None, None, amplitude * np.sin(2 * np.pi * t / period))
    })
    
    return model.solve()

In [ ]:
# solve the transient simulation
soln_time = 160
iterations = int(soln_time / model.timestep) + 1
record_interval = 10

# initialize
u0 = np.zeros((x.num_points, y.num_points, time_acc + 1))

# Run the simulation. This may take some time...
start = time.perf_counter()
result = []
for i in range(iterations):
    t = i * model.timestep
    res = update_timestep(u0, t)['u']
    
    avg_time = (time.perf_counter() - start) / (i + 1)
    
    print('\r', f'iteration: {i}    sim_time: {i * model.timestep:0.2f}s    max_val: {res.max():0.2f}   time/iter: {avg_time:0.4f}s    ETA: {avg_time * (iterations - i) / 60:0.2f}min', end='')
    
    if i%record_interval==0:
        result.append(res)
        
    # update the time history
    u0 = np.dstack([u0[:,:,1:], res])

In [ ]:
# plot some results!
hv.output(widget_location='bottom')

results = hv.HoloMap({
    i * model.timestep * record_interval: hv.Image(
        res[:, ::-1].T,
        bounds = (0, 0, 500 ,500)
    ).opts(
        cmap = 'Blues_r',
        tools = ['hover'], colorbar = True,
    )
    for i, res in enumerate(result)
}, kdims = 'Time (s)').opts(
    width = 800, height = 800,
    tools = ['hover'], show_grid = True,
    xlabel = 'x (m)', ylabel = 'y (m)'

)

In [ ]:
hv.save(results, 'shoreline_waves.html')